In [ ]:
# Efficient Frontier GUI with Export (Fully Corrected Version)import pandas as pdimport numpy as npfrom datetime import datetime, datefrom io import BytesIOfrom ipywidgets import (    FileUpload, Button, VBox, Output, Label, SelectMultiple, HBox,    FloatSlider, Dropdown)from IPython.display import display, clear_outputfrom pypfopt.efficient_frontier import EfficientFrontierfrom pypfopt import objective_functionsimport copyoutput_all = Output()df_global = Noneselected_assets = []sector_mapper = {}sector_constraint_widgets = {}upload = FileUpload(accept=".xlsx", multiple=False)def log_returns(df):    return np.log(df / df.shift(1)).dropna()def ewma_covariance_matrix(X, alpha):    T, D = X.shape    cov = np.zeros((D, D))    for tt in range(T):        x_t = X[tt, :]        if tt == 0:            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]        else:            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]    return covdef show_asset_selector():    with output_all:        clear_output()        asset_select = SelectMultiple(description="Assets:", options=df_global.columns.tolist(), rows=10)        continue_btn = Button(description="➡ Define Sectors", button_style="primary")        def proceed(change):            global selected_assets            selected_assets = list(asset_select.value)            if not selected_assets:                print("⚠️ Please select at least one asset.")                return            show_sector_mapping()        continue_btn.on_click(proceed)        display(VBox([            Label("📊 Step 2: Select assets:"),            asset_select,            continue_btn        ]))def show_sector_mapping():    with output_all:        clear_output()        mapping_widgets = {}        rows = []        for asset in selected_assets:            label = Label(value=asset, layout={"width": "200px"})            dropdown = Dropdown(                options=["Equities", "Corporate Bonds", "High Yield", "Euro Government Core", "Cash"],                layout={"width": "200px"}            )            mapping_widgets[asset] = dropdown            rows.append(HBox([label, dropdown]))        next_btn = Button(description="➡ Set Sector Constraints", button_style="primary")        def proceed_mapping(change):            global sector_mapper            sector_mapper = {a: w.value for a, w in mapping_widgets.items()}            show_sector_constraints()        next_btn.on_click(proceed_mapping)        display(VBox([            Label("🏷 Step 3: Assign sectors to assets:"),            VBox(rows),            next_btn        ]))def show_sector_constraints():    with output_all:        clear_output()        global sector_constraint_widgets        sector_constraint_widgets = {}        unique_sectors = set(sector_mapper.values())        sliders = []        for sector in unique_sectors:            lower = FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description=f"{sector} Min", layout={"width": "45%"})            upper = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description=f"{sector} Max", layout={"width": "45%"})            sector_constraint_widgets[sector] = (lower, upper)            sliders.append(HBox([lower, upper]))        gamma_slider = FloatSlider(value=0.01, min=0.0, max=0.1, step=0.005, description="L2 Gamma:")        run_button = Button(description="🚀 Run Optimizer", button_style="success")        result_output = Output()        def run_optimizer(change):            result_output.clear_output()            with result_output:                try:                    df = df_global[selected_assets]                    logret = log_returns(df).astype(np.float64)                    cov = ewma_covariance_matrix(logret.values, alpha=0.99)                    cov = pd.DataFrame(cov * 50, index=df.columns, columns=df.columns)                    mu = pd.Series({asset: 0.05 for asset in selected_assets})                    ef_template = EfficientFrontier(mu, cov, weight_bounds=(0, 1))                    ef_template.add_objective(objective_functions.L2_reg, gamma=gamma_slider.value)                    sector_lower = {s: pair[0].value for s, pair in sector_constraint_widgets.items()}                    sector_upper = {s: pair[1].value for s, pair in sector_constraint_widgets.items()}                    ef_template.add_sector_constraints(sector_mapper, sector_lower, sector_upper)                    min_ret = ef_template.expected_returns.min()                    max_ret = ef_template.expected_returns.max() - 0.001                    return_range = np.linspace(min_ret, max_ret, 25)                    weights = []                    metrics = []                    for r in return_range:                        ef = copy.deepcopy(ef_template)                        ef.efficient_return(r)                        weights.append(ef.weights)                        metrics.append(ef.portfolio_performance())                    weights_df = pd.DataFrame(weights, columns=mu.index)                    weights_df["Target Return"] = return_range                    metrics_df = pd.DataFrame(metrics, columns=["Mean", "Volatility", "Sharpe"])                    display(weights_df.head())                    display(metrics_df.head())                    today = date.today().strftime("%d%m%y")                    weights_df.to_excel(f"efficient_frontier_weights_{today}.xlsx", index=False)                    metrics_df.to_excel(f"efficient_frontier_metrics_{today}.xlsx", index=False)                    print(f"✅ Files saved: efficient_frontier_weights_{today}.xlsx, efficient_frontier_metrics_{today}.xlsx")                except Exception as e:                    print("❌ Error:", e)        run_button.on_click(run_optimizer)        display(VBox([            Label("📐 Step 4: Set sector constraints:"),            VBox(sliders),            gamma_slider,            run_button,            result_output        ]))def handle_upload(change):    global df_global    with output_all:        clear_output()        try:            file_info = upload.value[0]            content = BytesIO(file_info["content"])            df = pd.read_excel(content, sheet_name="PyData", engine="openpyxl")            df = df.rename(columns=df.iloc[0]).drop(0)            df["Dates"] = pd.to_datetime(df["Dates"])            df = df.set_index("Dates")            df = df.astype(float)            df = df[df.index > datetime(2017, 1, 1)]            df_global = df            print("✅ File loaded.")            show_asset_selector()        except Exception as e:            print("❌ Error loading file:", e)upload.observe(handle_upload, names="value")display(VBox([    Label("📥 Step 1: Upload Excel file (sheet: PyData):"),    upload,    output_all]))